In [1]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW
from pathlib import Path
from argparse import Namespace
import wandb
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(2)

In [3]:
from transformers import (
  AutoTokenizer,
  AutoModelForQuestionAnswering,
)

# model = AutoModelForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModelForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large").to(device)
tokenizer = AutoTokenizer.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")
# model = AutoModelForQuestionAnswering.from_pretrained("google-bert/bert-base-multilingual-uncased")
# tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

In [4]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("D://Code/Machine Translation/hw7_train.json")
dev_questions, dev_paragraphs = read_data("D://Code/Machine Translation/hw7_dev.json")
test_questions, test_paragraphs = read_data("D://Code/Machine Translation/hw7_test.json")

In [9]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

In [12]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60
        self.max_paragraph_len = 150
        
        ##### TODO: Change value of doc_stride #####
        # self.doc_stride = 150
        self.doc_stride = 75

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            # paragraph_end = paragraph_start + self.max_paragraph_len

            max_offset = self.max_paragraph_len / 4   # We allow up to 1/4 of the max length as offset
            random_offset = np.random.randint(-max_offset, max_offset)  # Random shift between -max_offset and +max_offset

            # Adjust paragraph start based on random offset
            paragraph_start = max(0, min(mid + random_offset - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

In [13]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        #fix the bug 
        if start_index <= end_index:
            if prob > max_prob:
                max_prob = prob
                # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
                answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
        
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

In [14]:
config = Namespace(
    #save the checkpoint in wandb
    savedir = "./checkpoints/bert-boss",

    # training epoch
    num_epoch = 3,
    
    #Validate on validation set
    validation = True,
    
    #print the loss per logging step
    logging_step = 100,
    
    #learning rate
    learning_rate = 1e-5,

    # training batch size 
    train_batch_size = 8,

    # logging
    use_wandb=True,
)

In [15]:
from accelerate import Accelerator
from transformers import get_linear_schedule_with_warmup

#### TODO: gradient_accumulation (optional)####
# Note: train_batch_size * gradient_accumulation_steps = effective batch size
# If CUDA out of memory, you can make train_batch_size lower and gradient_accumulation_steps upper
# Doc: https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation
gradient_accumulation_steps = 16
optimizer = AdamW(model.parameters(), lr=config.learning_rate)

# dataloader
# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=config.train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)


total_steps = len(train_loader) * config.num_epoch
num_warmup_steps = int(0.2 * total_steps)  # Set warmup steps to 20% of total steps
# [Hugging Face] Apply linear learning rate decay with warmup
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps
)

# Change "fp16_training" to True to support automatic mixed 
# precision training (fp16)	
fp16_training = True
if fp16_training:    
    accelerator = Accelerator(mixed_precision="fp16", gradient_accumulation_steps=gradient_accumulation_steps)
else:
    accelerator = Accelerator(gradient_accumulation_steps=gradient_accumulation_steps)
if config.use_wandb:
    wandb.init(project="Fintuning-QA-Bert", name=Path(config.savedir).stem, config=config)

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/
model, optimizer, train_loader, scheduler = accelerator.prepare(model, optimizer, train_loader, scheduler) 

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: baoxihuang0429 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
model.train()


print("Start Training ...")

for epoch in range(config.num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        with accelerator.accumulate(model):
            
            # Load all data into GPU
            data = [i.to(device) for i in data]
            
            # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
            # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
            output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
            # Choose the most probable start position / end position
            start_index = torch.argmax(output.start_logits, dim=1)
            end_index = torch.argmax(output.end_logits, dim=1)
            
            # Prediction is correct only if both start_index and end_index are correct
            train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
               
            train_loss += output.loss
            
            accelerator.backward(output.loss)
            
            step += 1
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        ##### TODO: Apply linear learning rate decay #####

        # Print training loss and accuracy over past logging step
        if step % config.logging_step == 0:
             if config.use_wandb:
                 wandb.log({
                    "train/loss": train_loss.item() / config.logging_step,
                    "train/acc": train_acc / config.logging_step,
                })
             print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / config.logging_step:.3f}, acc = {train_acc / config.logging_step:.3f}")
             train_loss = train_acc = 0

    if config.validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            if config.use_wandb: 
                wandb.log({
                    "Validation/acc": dev_acc / len(dev_loader),
                })
            print(f"Validation | Epoch {config.num_epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 2.334, acc = 0.411
Epoch 1 | Step 200 | loss = 2.113, acc = 0.445
Epoch 1 | Step 300 | loss = 1.698, acc = 0.519
Epoch 1 | Step 400 | loss = 1.320, acc = 0.592
Epoch 1 | Step 500 | loss = 1.182, acc = 0.610
Epoch 1 | Step 600 | loss = 1.047, acc = 0.634
Epoch 1 | Step 700 | loss = 0.910, acc = 0.660
Epoch 1 | Step 800 | loss = 0.911, acc = 0.642
Epoch 1 | Step 900 | loss = 0.796, acc = 0.681
Epoch 1 | Step 1000 | loss = 0.704, acc = 0.709
Epoch 1 | Step 1100 | loss = 0.636, acc = 0.745
Epoch 1 | Step 1200 | loss = 0.692, acc = 0.725
Epoch 1 | Step 1300 | loss = 0.619, acc = 0.738
Epoch 1 | Step 1400 | loss = 0.625, acc = 0.729
Epoch 1 | Step 1500 | loss = 0.689, acc = 0.724
Epoch 1 | Step 1600 | loss = 0.568, acc = 0.759
Epoch 1 | Step 1700 | loss = 0.604, acc = 0.759
Epoch 1 | Step 1800 | loss = 0.588, acc = 0.743
Epoch 1 | Step 1900 | loss = 0.549, acc = 0.752
Epoch 1 | Step 2000 | loss = 0.724, acc = 0.721
Epoch 1 | Step 2100 | loss = 0.588, acc = 0.745
E

  0%|          | 0/2863 [00:00<?, ?it/s]

Validation | Epoch 4 | acc = 0.785


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 2 | Step 100 | loss = 0.481, acc = 0.765


In [17]:
wandb.finish()

Validation/acc,▁▇█
train/acc,▁▁▁▁▁▁▁▂▅▅▆▆▆▇▆▇▇▇▇▇▇▇▇█▇███████▇██████▇
train/loss,███▇▇▆▆▄▄▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation/acc,0.69787
train/acc,0.63
train/loss,0.85947
